<a href="https://colab.research.google.com/github/mbridgets/TheMUGSSProject3/blob/master/loading_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-2.4.6'
spark_version = 'spark-2.4.7'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Get:12 https://cloud.r-project.org/b

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2020-12-17 03:22:17--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  5.02MB/s    in 0.2s    

2020-12-17 03:22:17 (5.02 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [4]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url="https://mugssproject3avodatageo.s3-us-west-1.amazonaws.com/avo_data_withlr.csv"
spark.sparkContext.addFile(url)
avo_geo_data_df = spark.read.csv(SparkFiles.get("avo_data_withlr.csv"), sep=",", header=True, inferSchema=True)

# Show DataFrame
avo_geo_data_df.show()

+-------------------+-------------+------------+---------+---------+---------+----------+----------+----------+-----------+------------+----+------+-----+---------+----------+--------+---------+----------------+
|               date|average_price|total_volume|type_4046|type_4225|type_4770|total_bags|small_bags|large_bags|xlarge_bags|        type|year|  city|state|   region|city_state|latitude|longitude|LinearRegression|
+-------------------+-------------+------------+---------+---------+---------+----------+----------+----------+-----------+------------+----+------+-----+---------+----------+--------+---------+----------------+
|2015-12-27 00:00:00|         1.33|    64236.62|  1036.74| 54454.85|    48.16|   8696.87|   8603.62|     93.25|        0.0|conventional|2015|Albany|   NY|Northeast|Albany, NY| 42.6664| -73.7987|            null|
|2015-12-20 00:00:00|         1.35|    54876.98|   674.28| 44638.81|    58.33|   9505.56|   9408.07|     97.49|        0.0|conventional|2015|Albany|   N

In [5]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://mugssproject3-awsdb.cd7njcklugjr.us-west-1.rds.amazonaws.com:5432/MUGSSProject3DB"
config = {"user":"MUGSSProject3DB", 
          "password": "data1234", 
          "driver":"org.postgresql.Driver"}


In [7]:
# Write DataFrame to active_user table in RDS

avo_geo_data_df.write.jdbc(url=jdbc_url, table='avocado_sales', mode=mode, properties=config)